# Ising model simulation with Monte Carlo method

In [ ]:
import os
import shutil
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

In [ ]:
fdir = './out/ising/'
ffmt = 'png'
fdpi = 150

## Characteristic parameters of the system

In [ ]:
k_B = 1.380648e-23  # Boltzmann constant
h = 0               # External magnetic field
J = 1               # Coupling parameter
T = 1               # Temperature
beta = 1 / T        # Dimensionless thermodynamic beta

In [ ]:
def get_random_spins(size):
    '''
    Returns an array filled with +1 and -1 values, randomly sampled from
    a uniform distribution.
    '''
    return np.sign(np.random.random(size) - 0.5)

In [ ]:
def e_i(s_i, s_j, J=1):
    '''
    Calculates the interaction energy between two spins.
    '''
    return -J * s_i * s_j

In [ ]:
def E(spins, J=1, h=0):
    '''
    Calculates the Hamiltonian for a given spin configuration.
    '''
    # Size of the lattice
    Nr, Nc = spins.shape[0], spins.shape[1]
    # Relative indexes of neighbouring spins
    nb = np.array((-1, -Nc, 1, Nc))

    E = 0
    for k, s_k in enumerate(spins.flat):
        # Flattened indexes
        k_nb = (k - nb) % (Nr*Nc)
        # Calculate the interactions between neighbouring spins
        E += np.sum(-J * s_k * spins.flat[k_nb]) / 4
        # Apply the external magnetic field
        E -= h*s_k
    return E

In [ ]:
def m(spins):
    return np.mean(spins)

In [ ]:
def delta_E(spins, i, j, J=1, h=0):
    '''
    Calculate the change in the Hamiltonian on the flip of the (i, j)
    spin.
    '''
    # Size of the lattice
    Nr, Nc = spins.shape[0], spins.shape[1]
    # Relative indexes of neighbouring spins
    nb = np.array((-1, -Nc, 1, Nc))

    # Flattened indexes
    k = i*Nc + j
    k_nb = (k - nb) % (Nr*Nc)

    dE = np.sum(spins.flat[k] * spins.flat[k_nb])

    return 2*J*dE + 2*h*spins.flat[k]

In [ ]:
def mcmc_step(spins, beta=0.01, J=1, h=0):

    # Choose random indexes
    i = np.random.randint(0, spins.shape[0])
    j = np.random.randint(0, spins.shape[1])

    # Calculate energy difference on flip of spin_ij
    dE = delta_E(spins, i, j, J, h)

    # If dE < 0, then execute the flip
    if(dE < 0):
        spins[i, j] *= -1
    # If dE > 0, choose a random number R, between 0 and 1
    # If R < e^(-beta * dE), then execute the spin flip
    # If not, then leave the spins' state untouched and continue with the next step
    elif(np.random.random() < np.exp(-beta * dE)):
        spins[i, j] *= -1

    return spins

In [ ]:
def run_simulation(spins, N_iter, T, J, h, save=False):

    # Calculate the dimensionless thermodynamic beta
    beta = 1 / T

    spins_n = spins.copy()
    for _ in range(N_iter):
        spins_n = mcmc_step(spins_n, beta=beta, J=J, h=h)

    if save:
        np.save(file=os.path.join(fdir, f'spins_T{T}'), arr=spins_n)

    return spins_n

In [ ]:
Ts = np.array([0.1, 1, 2.5, 5])
# Size of lattice
Nr, Nc = 100, 100
# The current positions of the spins
spins = get_random_spins((Nr, Nc))
# Number of iterations
N_iter = 500_000

if not os.path.exists(fdir):
    os.makedirs(fdir)

Ss = Parallel(n_jobs=4)(delayed(run_simulation)(spins, N_iter=N_iter, T=T, J=J, h=h, save=True) for T in Ts)

In [ ]:
nrows, ncols = 1, Ts.size
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*7, nrows*7),
                         facecolor='ghostwhite')
fig.subplots_adjust(wspace=0.0)

spin_files = {
  f.split('T')[1].split('.npy')[0] : f for f in os.listdir(fdir) if '.npy' in f
}
for T, ax in zip(Ts, axes.flat):
    ax.axis('off')

    spins_n = np.load(file=os.path.join(fdir, spin_files[f'{T}']))
    ax.imshow(spins_n, cmap='Greys',
            interpolation='none')
    ax.set_title(f'{T = } K\nSteps = {N_iter:,}',
                 fontsize=16, fontweight='bold')
plt.show()

## Temperature dependence of energy and magnetization

In [ ]:
Ts = np.linspace(0.01, 5, 10)
# Size of lattice
Nr, Nc = 50, 50
# The current positions of the spins
spins = get_random_spins((Nr, Nc))
# Number of iterations
N_iter = 500_000

Ss = np.zeros((len(Ts), Nr, Nc))
for i, T in enumerate(Ts):
    Ss[i] = run_simulation(spins, N_iter=N_iter, T=T, J=J, h=h, save=True)

In [ ]:
nr, nc = 2, 5
fig, axes = plt.subplots(nr, nc, figsize=(nc*4, nr*4))

for i, ax in enumerate(axes.flat):
    ax.axis('off')
    ax.imshow(Ss[i], cmap='Greys', interpolation='none')

plt.show()

In [ ]:
Es = np.zeros_like(Ts)
for i, s in enumerate(Ss):
    Es[i] = E(s, J=J, h=h)

ms = np.zeros_like(Ts)
for i, s in enumerate(Ss):
    ms[i] = m(s)

In [ ]:
nr, nc = 1, 2
fig, axes = plt.subplots(nr, nc, figsize=(nc*7, nr*5), dpi=120)
axes = axes.flat
for ax in axes:
    ax.grid(True, ls='--', alpha=0.6)

axes[0].scatter(Ts, Es, s=36)
axes[0].set_xlabel('Temperature', fontsize=16, fontweight='bold')
axes[0].set_ylabel('Energy', fontsize=16, fontweight='bold')

axes[1].scatter(Ts, ms, s=36)
axes[1].set_xlabel('Temperature', fontsize=16, fontweight='bold')
axes[1].set_ylabel('Magnetization', fontsize=16, fontweight='bold')

plt.show()